In [1]:
from src.time_series_prep import *
import pandas as pd
from models.personalized_model import *
from models.helper_functions import *
from torchviz import make_dot
from torchsummary import summary

from src.data_preprocessing import *
from src.visualizations import *
from src.global_configs import *
from src.time_series_prep import *
import logging
from src.post_processing import *
from collections import defaultdict
# Initialize logger
logging.basicConfig(
    filename='training.log',  # File where logs will be saved
    level=logging.INFO,  # Set logging level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log format
    # filemode='w'  # Overwrite log file on each run
)
logger = logging.getLogger()

In [2]:
model_type = 'personalized'

scaler, train_patients, test_patients = process_all_csv_files_personalized(input_folder, output_folder)

In [3]:

histories_withval_PHs =  {key: [] for key in prediction_horizons}
histories_noval_PHs = {key: [] for key in prediction_horizons}


MAE_Patients_PHs =  {key: [] for key in prediction_horizons}
RMSE_Patients_PHs =  {key: [] for key in prediction_horizons}


In [5]:
prediction_horizon = 6


for i, patient in enumerate(train_patients.items()):
    patient_code = patient[0]
    df = patient[1]

    print(f"Patient {patient_code}")
    # print(df)

    df_test = list(test_patients.values())[i]


    train_loader, validation_loader, all_train_loader, test_loader, input_shape, input_shape_test, output_shape, output_shape_test =  prepare_data_loader(
                                                                                        window_size,BATCH_SIZE, prediction_horizon,
                                                                                        model_type , split_ratio=0.8, df = df, df_test = df_test, output_folder_train = None, shuffle = True)
    model_all_train = PersonalizedModelWithAttention(input_shape =input_shape , output_shape=output_shape) 
    model_all_train = model_all_train.to(device)
    model_all_train, history2 = train_model(model_all_train, all_train_loader, val_loader = test_loader, epochs=150, learning_rate=0.00001, model_type = model_type)

    histories_noval_PHs[prediction_horizon] = history2
    break

Patient 559


Epoch 1/150:   1%|          | 1/189 [00:00<00:11, 16.95batch/s, mae_loss=0.893]

Epoch 1/150: 100%|██████████| 189/189 [00:04<00:00, 38.40batch/s, mae_loss=0.578]


Epoch [1/150], Training MAE: 0.6301, Training RMSE: 0.7987
Epoch [1/150], Validation MAE: 0.4231, Validation RMSE: 0.5739


Epoch 2/150: 100%|██████████| 189/189 [00:04<00:00, 38.21batch/s, mae_loss=0.372]


...

Epoch [148/150], Training MAE: 0.2719, Training RMSE: 0.3954
Epoch [148/150], Validation MAE: 0.2071, Validation RMSE: 0.3211


Epoch 149/150: 100%|██████████| 189/189 [00:03<00:00, 48.05batch/s, mae_loss=0.356]


Epoch [149/150], Training MAE: 0.2707, Training RMSE: 0.3962
Epoch [149/150], Validation MAE: 0.2064, Validation RMSE: 0.3201


Epoch 150/150: 100%|██████████| 189/189 [00:04<00:00, 47.16batch/s, mae_loss=0.238]


Epoch [150/150], Training MAE: 0.2700, Training RMSE: 0.3950
Epoch [150/150], Validation MAE: 0.2079, Validation RMSE: 0.3201


In [6]:
pd.DataFrame(histories_noval_PHs[6]).to_csv("saved_models\\loss_ph6_personalized.csv")

In [ ]:
for prediction_horizon in prediction_horizons:
  logger.info(f"Start Prediction Horizon {prediction_horizon} of {model_type} model")
  print(f"Start Prediction Horizon {prediction_horizon} of {model_type} model")


  for i, patient in enumerate(train_patients.items()):
    patient_code = patient[0]
    df = patient[1]
    logger.info(f"Patient {patient_code}")
    print(f"Patient {patient_code}")

    df_test = list(test_patients.values())[i]
    

    train_loader, validation_loader, all_train_loader, test_loader, input_shape, input_shape_test, output_shape, output_shape_test =  prepare_data_loader(
                                                                                        window_size,BATCH_SIZE, prediction_horizon,
                                                                                        model_type , split_ratio=0.8, df = df, df_test = df_test, output_folder_train = None, shuffle = True)
    for r in range(5):
        model_all_train = PersonalizedModelWithAttention(input_shape =input_shape , output_shape=output_shape) 
        model_all_train = model_all_train.to(device)
        model_all_train, history2 = train_model(model_all_train, all_train_loader, val_loader = test_loader, epochs=100, learning_rate=0.00001, model_type = model_type)

        model_all_train.eval()

        torch.save(model_all_train.state_dict(), f'saved_models//model_{model_type}_{str(prediction_horizon)}_{str(patient_code)}_{r}.pth')
        
        print(f"model_{model_type}_{str(prediction_horizon)}_{str(patient_code)}_{r} saved")
        logger.info(f"model_{model_type}_{str(prediction_horizon)}_{str(patient_code)}_{r} saved")

  logger.info(f"End Prediction Horizon of {prediction_horizon} of {model_type} Model")
  print(f"End Prediction Horizon of {prediction_horizon} of {model_type} Model")



In [4]:
abs_patients_errors_PHs =  {key: [] for key in prediction_horizons}
squared_patients_errors_PHs =  {key: [] for key in prediction_horizons}

for prediction_horizon in prediction_horizons:
    all_test_mae =  {key: [] for key in train_patients.keys()}
    all_test_rmse =  {key: [] for key in train_patients.keys()}
    patient_mae = []
    patient_rmse = []

    for i, patient in enumerate(train_patients.items()):
        patient_code = patient[0]
        df = patient[1]
        df_test = list(test_patients.values())[i]
        

        train_loader, validation_loader, all_train_loader, test_loader, input_shape, input_shape_test, output_shape, output_shape_test =  prepare_data_loader(
                                                                                            window_size,BATCH_SIZE, prediction_horizon,
                                                                                            model_type , split_ratio=0.8, df = df, df_test = df_test, output_folder_train = None, shuffle = False)
        multiple_runs_each_patient_mae = []
        multiple_runs_each_patient_rmse = []
        for r in range(5):
            model_all_train_read =  PersonalizedModelWithAttention(input_shape =input_shape , output_shape=output_shape)
            model_all_train_read.load_state_dict(torch.load(f'saved_models//model_{model_type}_{str(prediction_horizon)}_{str(patient_code)}_{r}.pth',weights_only=False))
            model_all_train_read.to(device)
            model_all_train_read.eval()
            each_patient_mae, each_patient_rmse= evaluate_test(model_all_train_read,test_loader, device ,scaler, mask_value, model_type = model_type)
            
            multiple_runs_each_patient_mae.append(each_patient_mae)
            multiple_runs_each_patient_rmse.append(each_patient_rmse)
        patient_mae.append(multiple_runs_each_patient_mae)
        patient_rmse.append(multiple_runs_each_patient_rmse)
        all_test_mae[patient_code] = multiple_runs_each_patient_mae
        all_test_rmse[patient_code] = multiple_runs_each_patient_rmse

    abs_patients_errors_PHs[prediction_horizon] = all_test_mae
    squared_patients_errors_PHs[prediction_horizon] = all_test_rmse


In [ ]:
abs_patients_errors_PHs_df = pd.DataFrame()
mean_ph = []
for i, patient_mae in enumerate(abs_patients_errors_PHs.values()):

    grouped_values = defaultdict(list)

    # Group values by keys
    for d in abs_patients_errors_PHs[prediction_horizons[i]].items():
        grouped_values[d[0]].append(d[1])

    test_list = [values[0] for values in grouped_values.values()]
    

    mean_values = {key: np.mean(values) for key, values in grouped_values.items()}
    std_values = {key: np.std(values, ddof=1) for key, values in grouped_values.items()}


    mean_std = [f"{mean_values[i]:.2f} ± {std_values[i]:.2f}" for i in patients_list]
    abs_patients_errors_PHs_df[prediction_horizons[i]] = mean_std

    mean_all_patients = np.mean(list(mean_values.values()))
    mean_all_patients = [f"{mean_values[i]:.2f} ± {std_values[i]:.2f}"for i in patients_list]
    
    mean_values = {key: np.mean(values) for key, values in grouped_values.items()}


    mean_all = [np.mean(patient_mae[i]) for i in patients_list]


    std_each_run = np.std([np.mean([one[i] for one in test_list]) for i in range(5)])
    mean_std_all = [f"{np.mean(mean_all) :.2f} ± {std_each_run:.2f}" for i in range(5)]

    mean_ph.append(mean_std_all[0])

abs_patients_errors_PHs_df.loc[len(abs_patients_errors_PHs_df.index)] = mean_ph
abs_patients_errors_PHs_df.index = patients_list+["Mean"]

abs_patients_errors_PHs_df

In [ ]:
squared_patients_errors_PHs_df = pd.DataFrame()
mean_ph = []
for i, patient_mae in enumerate(squared_patients_errors_PHs.values()):

    grouped_values = defaultdict(list)

    # Group values by keys
    for d in squared_patients_errors_PHs[prediction_horizons[i]].items():
        grouped_values[d[0]].append(d[1])

    test_list = [values[0] for values in grouped_values.values()]
    


    mean_values = {key: np.mean(values) for key, values in grouped_values.items()}
    std_values = {key: np.std(values, ddof=1) for key, values in grouped_values.items()}


    mean_std = [f"{mean_values[i]:.2f} ± {std_values[i]:.2f}" for i in patients_list]
    squared_patients_errors_PHs_df[prediction_horizons[i]] = mean_std

    mean_all_patients = np.mean(list(mean_values.values()))
    mean_all_patients = [f"{mean_values[i]:.2f} ± {std_values[i]:.2f}"for i in patients_list]
    
    mean_values = {key: np.mean(values) for key, values in grouped_values.items()}


    mean_all = [np.mean(patient_mae[i]) for i in patients_list]

 
    std_each_run = np.std([np.mean([one[i] for one in test_list]) for i in range(5)])
    mean_std_all = [f"{np.mean(mean_all) :.2f} ± {std_each_run:.2f}" for i in range(5)]

    mean_ph.append(mean_std_all[0])


squared_patients_errors_PHs_df.loc[len(squared_patients_errors_PHs_df.index)] = mean_ph
squared_patients_errors_PHs_df.index = patients_list+["Mean"]

squared_patients_errors_PHs_df

In [ ]:


# Instantiate the model for 12 tasks
input_shape = (64, 12, 1)  # Example input shape: batch_size, sequence_length, input_channels
output_tasks = 1           # Number of tasks

# Instantiate the model
model = PersonalizedModelWithAttention(input_shape=input_shape, output_shape=(output_tasks,))
model.to(device)
# Count the parameters
total_params = count_parameters(model)
print(f"Total parameters in the model with {output_tasks} tasks: {total_params}")


In [ ]:
# 170177 * 12 
# 2,042,124

In [ ]:
for i in histories_noval_PHs.items():
    print(f"Prediction horizon {i[0]}")
    plot_loss_curves(i[1])
    break